In [31]:
import cv2
import numpy as np

# Define marker ids and corners
marker_ids = []
marker_corners = []

# Define ArUco dictionary and detector parameters
dictionary = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_250)
parameters = cv2.aruco.DetectorParameters()

# Load image and detect markers
try:
    image = cv2.imread('./Room_with_ArUco_Markers-20230403/aruco_marker_2.jpg')
except:
    print("Error: Image file not found.")
else:
    marker_corners, marker_ids, rejected = cv2.aruco.detectMarkers(image, dictionary, parameters=parameters)
    print("Marker IDs:", marker_ids)

    # Create a copy of the original image
    output_image = image.copy()

    # Load and resize poster image
    try:
        poster = cv2.imread('./poster.jpg')
    except:
        print("Error: Poster file not found.")
    else:
        # Define the corners of the poster in its own coordinate system
        poster_corners = np.array([[image.shape[1] / 2 - poster.shape[1] * 0.2, image.shape[0] / 2 - poster.shape[0] * 0.2],
                           [image.shape[1] / 2 + poster.shape[1] * 0.2, image.shape[0] / 2 - poster.shape[0] * 0.2],
                           [image.shape[1] / 2 + poster.shape[1] * 0.2, image.shape[0] / 2 + poster.shape[0] * 0.2],
                           [image.shape[1] / 2 - poster.shape[1] * 0.2, image.shape[0] / 2 + poster.shape[0] * 0.2]])
        
        print('Poster Corner points: ', poster_corners)
 
        if marker_ids is not None:
            for i in range(len(marker_ids)):
                # Extract the four corners of the marker
                marker_corner_points = marker_corners[i][0]

                print('Marker Corner points: ', marker_corner_points)

                # Compute the homography matrix
                H, _ = cv2.findHomography(poster_corners, marker_corner_points)

                print ('Homography Matrix: ', H)

                # Resize the warped poster
                warped_poster = cv2.warpPerspective(poster, H, (image.shape[1], image.shape[0]), flags=cv2.INTER_CUBIC)


                # Perform element-wise multiplication of warped poster and original image
                mask = cv2.cvtColor(warped_poster, cv2.COLOR_BGR2GRAY)
                mask = cv2.threshold(mask, 0, 255, cv2.THRESH_BINARY)[1]
                mask = cv2.merge([mask, mask, mask])
                poster_on_marker = cv2.bitwise_and(warped_poster, mask)

                image_without_marker = cv2.bitwise_and(image, cv2.bitwise_not(mask))

                merged_image = cv2.add(image_without_marker, poster_on_marker)

            
            image = cv2.resize(image, (500, 700))
            cv2.imshow('Input image with marker', image)
            cv2.waitKey(0)
            cv2.destroyAllWindows() # close all windows 
            
            poster = cv2.resize(poster, (350, 500))
            cv2.imshow('Poster image', poster)
            cv2.waitKey(0)
            cv2.destroyAllWindows() # close all windows
            
            warped_poster = cv2.resize(warped_poster, (500, 700))
            cv2.imshow('Poster warped according to the perspective of the marker', warped_poster)
            cv2.waitKey(0)
            cv2.destroyAllWindows() # close all windows 

            image_without_marker = cv2.resize(image_without_marker, (500, 700))
            cv2.imshow('image without marker', image_without_marker)
            cv2.waitKey(0)
            cv2.destroyAllWindows() # close all windows
            
            poster_on_marker = cv2.resize(warped_poster, (500, 700))
            cv2.imshow('Poster on the marker', poster_on_marker)
            cv2.waitKey(0)
            cv2.destroyAllWindows() # close all windows 
             
            merged_image = cv2.resize(merged_image, (500, 700))
            cv2.imshow('merged image', merged_image)
            cv2.waitKey(0)
            cv2.destroyAllWindows() # close all windows

        else:
            print("No markers found in the image!!")


Marker IDs: [[0]]
Poster Corner points:  [[1129.2 1505.6]
 [2338.8 1505.6]
 [2338.8 3118.4]
 [1129.2 3118.4]]
Marker Corner points:  [[1855. 1952.]
 [1911. 1951.]
 [1912. 2009.]
 [1855. 2010.]]
Homography Matrix:  [[ 4.50679495e-02 -1.98419147e-02  1.80359420e+03]
 [-1.29291786e-03  1.38732956e-02  1.90059420e+03]
 [-2.45895371e-07 -1.06964500e-05  1.00000000e+00]]
